In [1]:
%pip install requests

In [2]:
%load_ext autoreload
%autoreload 2

In [40]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
# authenticate
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

In [6]:
import proballer as pb

## Players

In [13]:
# df_players = pd.read_excel('/content/drive/My Drive/Colab Notebooks/aus_boomers_dev.xlsx')
df_players = pd.read_excel('/content/drive/My Drive/Colab Notebooks/AusBoomers.xlsx')

In [14]:
df_players.head()

,Name,ProballerID,Position,Shoots,Height,Weight,DOB,G,PTS,TRB,AST,FG%,FG3%,FT%,eFG%,PER,WS,Comp
0,Mitch Creek,7803,SF,R,196,97.0,1992-04-27,5.0,4.2,2.4,1.2,50.0,0.0,71.4,50.0,20.2,0.2,NBA
1,Ben Simmons,56765,PG,L,211,108.0,1996-07-20,242.0,16.3,8.3,8.0,56.0,10.0,60.1,56.1,20.1,27.2,NBA
2,Patty Mills,47394,PG,R,185,81.0,1988-08-11,698.0,8.9,1.7,2.3,43.2,39.0,85.6,54.1,14.5,31.8,NBA
3,Aron Baynes,37144,C,R,208,117.0,1986-12-09,494.0,5.9,4.6,0.8,49.0,30.9,80.0,51.0,13.8,19.4,NBA
4,Joe Ingles,47393,SF,L,203,NaN,1987-10-02,503.0,8.5,3.2,3.7,45.1,41.0,76.1,58.0,12.7,30.7,NBA


In [15]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Name         24 non-null     object        
 1   ProballerID  24 non-null     int64         
 2   Position     24 non-null     object        
 3   Shoots       16 non-null     object        
 4   Height       24 non-null     int64         
 5   Weight       13 non-null     float64       
 6   DOB          24 non-null     datetime64[ns]
 7   G            13 non-null     float64       
 8   PTS          13 non-null     float64       
 9   TRB          13 non-null     float64       
 10  AST          13 non-null     float64       
 11  FG%          13 non-null     float64       
 12  FG3%         13 non-null     float64       
 13  FT%          13 non-null     float64       
 14  eFG%         13 non-null     float64       
 15  PER          13 non-null     float64       
 16  WS        

## Methodology

Each player has an id.  Each page has one or more tables.  

- First table is 'regular' season.  
- Subsequent tables for playoff, European, or national games, are in tabbed panels.

In [16]:
df0 = pd.DataFrame()

for index, row in df_players.iterrows():

  id = row['ProballerID']
  name = row['Name']

  url = pb.get_player_url(str(id),name)

  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser') 

  # first pass
  df1 = pb.get_primary_table(soup,name)

  # second pass
  df2 = pb.get_secondary_tables(soup,name)

  df0 = df0.append(df1,ignore_index=True)
  df0 = df0.append(df2,ignore_index=True)





--- https://www.proballers.com/basketball/player/7803/mitch-creek ---
list-regular, 3
list-playoff, 1
list-international-competitions, 1
--- https://www.proballers.com/basketball/player/56765/ben-simmons ---
list-regular, 3
list-playoff, 1
list-international-competitions, 1
--- https://www.proballers.com/basketball/player/47394/patty-mills ---
list-regular, 3
list-playoff, 1
list-international-competitions, 1
--- https://www.proballers.com/basketball/player/37144/aron-baynes ---
list-regular, 4
list-playoff, 1
list-european-competitions, 1
list-international-competitions, 1
--- https://www.proballers.com/basketball/player/47393/joe-ingles ---
list-regular, 4
list-playoff, 1
list-european-competitions, 1
list-international-competitions, 1
--- https://www.proballers.com/basketball/player/65904/thon-maker ---
list-regular, 2
list-playoff, 1
--- https://www.proballers.com/basketball/player/61074/ryan-broekhoff ---
list-regular, 4
list-playoff, 1
list-european-competitions, 1
list-internati

In [51]:
# check for non-numeric columns
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Season  356 non-null    object 
 1   Age     356 non-null    float64
 2   League  356 non-null    object 
 3   Team    356 non-null    object 
 4   GP      356 non-null    float64
 5   MIN     356 non-null    float64
 6   FG%     356 non-null    object 
 7   3%      356 non-null    object 
 8   1%      356 non-null    object 
 9   MIN.1   356 non-null    float64
 10  Dr      356 non-null    float64
 11  Reb     356 non-null    float64
 12  Ast     356 non-null    float64
 13  Stl     356 non-null    float64
 14  Blk     356 non-null    float64
 15  Pts     356 non-null    float64
 16  Eff     356 non-null    float64
 17  Source  356 non-null    object 
 18  Name    356 non-null    object 
dtypes: float64(11), object(8)
memory usage: 53.0+ KB


In [52]:
# replace character with NaN
df = df0.replace('-',np.nan)

In [54]:
# convert columns to numeric
df[["FG%", "3%", "1%"]] = df[["FG%", "3%", "1%"]].apply(pd.to_numeric)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356 entries, 0 to 355
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Season  356 non-null    object 
 1   Age     356 non-null    float64
 2   League  356 non-null    object 
 3   Team    356 non-null    object 
 4   GP      356 non-null    float64
 5   MIN     356 non-null    float64
 6   FG%     354 non-null    float64
 7   3%      325 non-null    float64
 8   1%      330 non-null    float64
 9   MIN.1   356 non-null    float64
 10  Dr      356 non-null    float64
 11  Reb     356 non-null    float64
 12  Ast     356 non-null    float64
 13  Stl     356 non-null    float64
 14  Blk     356 non-null    float64
 15  Pts     356 non-null    float64
 16  Eff     356 non-null    float64
 17  Source  356 non-null    object 
 18  Name    356 non-null    object 
dtypes: float64(14), object(5)
memory usage: 53.0+ KB


In [56]:
df.to_excel('aus_boomers_proballer_nan.xlsx',index=False)